# 4 Extract object data

Using YOLOv8 for object detection to track objects in the videos. This will help identify toys and props used in baby joke interactions.


In [ ]:
import os
import math
import sys
import time
import pandas as pd
import numpy as np
import cv2
import torch
from ultralytics import YOLO

import sys

project_root = os.path.join("..")
sys.path.append(project_root)

from src.processors.face_processor import extract_faces_from_video, get_facial_stats
from src.utils.io_utils import getProcessedVideos, saveProcessedVideos

In [ ]:
# Add these to your imports
from src.config import PATH_CONFIG
from src.utils.notebook_utils import display_config_info, ensure_dir_exists

# Get paths from config
videos_in = PATH_CONFIG['videos_in']
data_out = PATH_CONFIG['data_out']

# Ensure output directory exists
if ensure_dir_exists(data_out):
    print(f"Created output directory: {data_out}")

# Display configuration information
display_config_info(videos_in, data_out, "Processing Configuration")

processedvideos = getProcessedVideos(data_out)
processedvideos.head()

In [ ]:
# Load YOLOv8 model for object detection
object_model = YOLO('yolov8n.pt')  # Uses the nano model for object detection



In [ ]:
# Add Objects columns to processedvideos if they don't exist
from src.processors.object_processor import extract_objects_from_video, match_objects_to_persons, normalize_object_coordinates


if "Objects.file" not in processedvideos.columns:
    processedvideos["Objects.file"] = None
if "Objects.when" not in processedvideos.columns:
    processedvideos["Objects.when"] = None
if "Objects.normed" not in processedvideos.columns:
    processedvideos["Objects.normed"] = None
if "Objects.matched" not in processedvideos.columns:
    processedvideos["Objects.matched"] = None

# Process each video for object detection
force_process = False

for index, row in processedvideos.iterrows():
    #testing only on first 5 videos
    if index > 5:
        break
    if force_process or pd.isnull(row["Objects.file"]):
        try:
            # Get video path
            video_path = os.path.join(videos_in, row["VideoID"])
            
            # Extract objects
            objects_df = extract_objects_from_video(video_path, object_model)
            
            # Save objects data
            stemname = os.path.splitext(row["VideoID"])[0]
            objects_path = os.path.join(data_out, f"{stemname}.objects.csv")
            objects_df.to_csv(objects_path, index=False)
            
            # Update record
            processedvideos.at[index, "Objects.file"] = objects_path
            processedvideos.at[index, "Objects.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
            
            # Normalize coordinates
            if len(objects_df) > 0:
                normed_df = normalize_object_coordinates(objects_df, row["Height"], row["Width"])
                normed_path = os.path.join(data_out, f"{stemname}.objects_normed.csv")
                normed_df.to_csv(normed_path, index=False)
                processedvideos.at[index, "Objects.normed"] = normed_path
            
            # Match objects to persons if keypoints exist
            if not pd.isnull(row["Keypoints.normed"]) and len(objects_df) > 0:
                poses_df = pd.read_csv(row["Keypoints.normed"])
                matched_df = match_objects_to_persons(normed_df, poses_df)
                matched_path = os.path.join(data_out, f"{stemname}.objects_matched.csv")
                matched_df.to_csv(matched_path, index=False)
                processedvideos.at[index, "Objects.matched"] = matched_path
            
            print(f"Processed objects for {row['VideoID']}")
        except Exception as e:
            print(f"Error processing objects for {row['VideoID']}: {e}")
    else:
        print(f"Already processed objects for {row['VideoID']}")

saveProcessedVideos(processedvideos, data_out)
processedvideos[['VideoID', 'Objects.file', 'Objects.normed', 'Objects.matched']].head()